In [1]:
import matplotlib.pyplot as plt
plt.rcParams['figure.facecolor'] = 'white'

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.wcs import WCS
from scipy.ndimage import median_filter
from ipywidgets import interact, widgets

fits_name_12co = "/Users/naoj306/Desktop/DB/SFP/AquilaRift_12CO_21.7arcsec_vel0.1_sph_v1.0.fits"
fits_name_13co = "/Users/naoj306/Desktop/DB/SFP/AquilaRift_13CO_22.1arcsec_vel0.1_sph_v1.0.fits"

with fits.open(fits_name_12co) as hdul_12co:
    data_12co = hdul_12co[0].data
    header_12co = hdul_12co[0].header

with fits.open(fits_name_13co) as hdul_13co:
    data_13co = hdul_13co[0].data
    header_13co = hdul_13co[0].header

velocities = (np.arange(header_12co['NAXIS3']) - (header_12co['CRPIX3']-1)) * header_12co['CDELT3'] / 1000.0 + header_12co['CRVAL3'] / 1000.0  # km/s


def plot_vy_slice(x_channel):
    plt.figure(figsize=(20, 8))

    # Data for each molecule type
    data_dict = {
        "12CO": data_12co[:, :, x_channel],
        "13CO": data_13co[:, :, x_channel]
    }
    
    # Create plots for each molecule type
    for i, (label, data) in enumerate(data_dict.items(), 1):
        filtered_data = median_filter(data, size=(50,3))
        diff_data = data - filtered_data

        # Original data
        tmax = np.nanmax(data)
        ax = plt.subplot(2, 4, i * 4 - 3)
        im = ax.imshow(data.T, origin='lower', aspect='auto', cmap='nipy_spectral', vmin=-tmax/3, vmax=tmax, extent=[velocities[0], velocities[-1], 0, data.shape[0]])
        plt.colorbar(im, ax=ax, label='Temperature (K)')
        plt.title(f'{label} VY Slice')

        # Median filtered data
        ax = plt.subplot(2, 4, i * 4 - 2)
        im = ax.imshow(filtered_data.T, origin='lower', aspect='auto', cmap='nipy_spectral', vmin=-tmax/3, vmax=tmax, extent=[velocities[0], velocities[-1], 0, data.shape[0]])
        plt.colorbar(im, ax=ax, label='Temperature (K)')
        plt.title(f'Median Filtered {label}')

        # Difference data
        tmax = np.nanmax(diff_data)
        ax = plt.subplot(2, 4, i * 4 - 1)
        im = ax.imshow(diff_data.T, origin='lower', aspect='auto', cmap='seismic', vmin=-tmax, vmax=tmax, extent=[velocities[0], velocities[-1], 0, data.shape[0]])
        plt.colorbar(im, ax=ax, label='Temperature (K)')
        plt.title('Difference')

        # Blank panel, use it for 'difference data' as placeholder
        ax = plt.subplot(2, 4, i * 4)
        im = ax.imshow(diff_data.T, origin='lower', aspect='auto', cmap='nipy_spectral',extent=[velocities[0], velocities[-1], 0, data.shape[0]])
        plt.colorbar(im, ax=ax, label='Temperature (K)')
        plt.title('blank')

    plt.tight_layout()  # Ensure subplots do not overlap
    plt.show()

interact(plot_vy_slice,
         x_channel=widgets.IntSlider(min=0, max=data_12co.shape[2]-1, step=1, value=data_12co.shape[2]//2, description='X channel'))


interactive(children=(IntSlider(value=267, description='X channel', max=533), Output()), _dom_classes=('widget…

<function __main__.plot_vy_slice(x_channel)>

In [59]:
from pycupid import clumpfind

def plot_vy_slice(x_channel):
    plt.figure(figsize=(20, 8))

    # Data for each molecule type
    data_dict = {
        "12CO": data_12co[:, :, x_channel],
        "13CO": data_13co[:, :, x_channel]
    }
    
    # Create plots for each molecule type
    for i, (label, data) in enumerate(data_dict.items(), 1):
        filtered_data = median_filter(data, size=(30,30))
        diff_data = data - filtered_data

        # Original data
        tmax = np.nanmax(data)
        ax = plt.subplot(2, 4, i * 4 - 3)
        im = ax.imshow(data.T, origin='lower', aspect='auto', cmap='nipy_spectral', vmin=-tmax/3, vmax=tmax, extent=[velocities[0], velocities[-1], 0, data.shape[0]])
        plt.colorbar(im, ax=ax, label='Temperature (K)')
        plt.title(f'{label} VY Slice')

        # Median filtered data
        ax = plt.subplot(2, 4, i * 4 - 2)
        im = ax.imshow(filtered_data.T, origin='lower', aspect='auto', cmap='nipy_spectral', vmin=-tmax/3, vmax=tmax,  extent=[velocities[0], velocities[-1], 0, data.shape[0]])
        plt.colorbar(im, ax=ax, label='Temperature (K)')
        plt.title(f'Median Filtered {label}')

        # Difference data
        print(np.nanmax(diff_data), np.nanmin(diff_data))
        tmax = np.nanmax(diff_data)
        ax = plt.subplot(2, 4, i * 4 - 1)
        im = ax.imshow(diff_data.T, origin='lower', aspect='auto', cmap='seismic', vmin=-tmax, vmax=tmax, extent=[velocities[0], velocities[-1], 0, data.shape[0]])
        plt.colorbar(im, ax=ax, label='Temperature (K)')
        plt.title('Difference')

        # Clumpfind data
        ax = plt.subplot(2, 4, i * 4)
        threshold = 0.2
        cf_result = clumpfind(diff_data, threshold)
        if cf_result is not None:
            cf = cf_result.astype("float32")
            cf_2 = cf.reshape(cf.shape[::-1])
            im = ax.imshow(cf_2, origin='lower', aspect='auto', cmap='jet', extent=[velocities[0], velocities[-1], 0, data.shape[0]])
            plt.colorbar(im, ax=ax, label='Clumpfind Result')
            plt.title('Clumpfind Result Map')
        else:
            plt.title('No Clumps Found')

    plt.tight_layout()  # Ensure subplots do not overlap
    plt.show()

interact(plot_vy_slice,
         x_channel=widgets.IntSlider(min=0, max=data_12co.shape[2]-1, step=1, value=data_12co.shape[2]//2, description='X channel'))


interactive(children=(IntSlider(value=267, description='X channel', max=533), Output()), _dom_classes=('widget…

<function __main__.plot_vy_slice(x_channel)>

In [5]:
from scipy.ndimage import median_filter
import numpy as np
from astropy.io import fits

# Assuming data_12co and data_13co are already loaded as shown in your initial code

x_channel = 360
fits_name_12co = "/Users/naoj306/Desktop/DB/SFP/AquilaRift_12CO_21.7arcsec_vel0.1_sph_v1.0.fits"

# Process the data for 12CO and 13CO
data_12co_slice = data_12co[:, :, x_channel]


# Median filtering
filtered_data_12co = median_filter(data_12co_slice, size=(1,99))


# Compute the difference data
diff_data_12co = data_12co_slice - filtered_data_12co


# Save the difference data to a FITS file
hdu_12co = fits.PrimaryHDU(diff_data_12co)

hdul_12co = fits.HDUList([hdu_12co])


# Write the FITS file
hdul_12co.writeto('diff_data_12CO_x360_0199.fits', overwrite=True)


print("Diff_data saved for 12CO and 13CO at x_channel 360.")


Diff_data saved for 12CO and 13CO at x_channel 360.


Diff_data saved for 12CO and 13CO at x_channel 360.
